In [171]:
# You can edit the font size here to make rendered text more comfortable to read
# It was built on a 13" retina screen with 18px
from IPython.core.display import display, HTML
display(HTML("<style>.rendered_html { font-size: 18px; }</style>"))

In [172]:
%matplotlib notebook
import sys
import os
from collections import Counter, OrderedDict
import numpy as np
from operator import itemgetter
import matplotlib.pyplot as plt
from astropy.table import Table
import schwimmbad
from cesium.time_series import TimeSeries
import cesium.featurize as featurize
from tqdm import tnrange, tqdm_notebook
import sklearn 
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [173]:
pbmap = OrderedDict([(0,'u'), (1,'g'), (2,'r'), (3,'i'), (4, 'z'), (5, 'y')])

# it also helps to have passbands associated with a color
pbcols = OrderedDict([(0,'blueviolet'), (1,'green'), (2,'red'),\
                      (3,'orange'), (4, 'black'), (5, 'brown')])

pbnames = list(pbmap.values())

In [174]:
datadir = 'data/'
metafilename = f'{datadir}/training_set_metadata.csv'
metadata = Table.read(metafilename, format='csv')
nobjects = len(metadata)
#metadata

In [175]:
counts = Counter(metadata['target'])
labels, values = zip(*sorted(counts.items(), key=itemgetter(1)))
fig, ax = plt.subplots()

cmap = plt.cm.tab20
nlines = len(labels)
classcolor =  list(cmap(np.linspace(0,1,nlines)))[::-1]

# we'll create a mapping between class and color
classcolmap = dict(zip(labels, classcolor))

indexes = np.arange(nlines)
width = 1
ax.bar(indexes, values, width, edgecolor='k',\
       linewidth=1.5, tick_label=labels, log=True, color=classcolor)
ax.set_xlabel('Target')
ax.set_ylabel('Number of Objects')
fig.tight_layout()

<IPython.core.display.Javascript object>

In [176]:
lcfilename = f'{datadir}/training_set.csv'
lcdata = Table.read(lcfilename, format='csv')
#lcdata

In [177]:
tsdict = OrderedDict()
for i in tnrange(nobjects, desc='Building Timeseries'):
    row = metadata[i]
    thisid = row['object_id']
    target = row['target']
    
    meta = {'z':row['hostgal_photoz'],\
            'zerr':row['hostgal_photoz_err'],\
            'mwebv':row['mwebv']}
    
    ind = (lcdata['object_id'] == thisid)
    thislc = lcdata[ind]

    pbind = [(thislc['passband'] == pb) for pb in pbmap]
    t = [thislc['mjd'][mask].data for mask in pbind ]
    m = [thislc['flux'][mask].data for mask in pbind ]
    e = [thislc['flux_err'][mask].data for mask in pbind ]

    tsdict[thisid] = TimeSeries(t=t, m=m, e=e,\
                        label=target, name=thisid, meta_features=meta,\
                        channel_names=pbnames )
    
del lcdata

In [178]:
features_to_use = ["amplitude",
                   "percent_beyond_1_std",
                   "maximum",
                   "max_slope",
                   "median",
                   "median_absolute_deviation",
                   "percent_close_to_median",
                   "minimum",
                   "skew",
                   "std",
                   "weighted_average"]

In [179]:
# we'll turn off warnings for a bit, because numpy can be whiny. 
import warnings
warnings.simplefilter('ignore')

In [180]:
featurefile = f'{datadir}/plasticc_featuretable.npz'
if os.path.exists(featurefile):
    featuretable, _ = featurize.load_featureset(featurefile)
else:
    print("featuretable not found")
    #to create featuretable use demo nb
            
    featuretable = featurize.assemble_featureset(features_list=features_list,\
                              time_series=tsdict.values())
    featurize.save_featureset(fset=featuretable, path=featurefile)

In [181]:
old_names = featuretable.columns.values
new_names = ['{}_{}'.format(x, pbmap.get(y,'meta')) for x,y in old_names]
cols = [featuretable[col] for col in old_names]
allfeats = Table(cols, names=new_names)
del featuretable

In [182]:
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
splits = list(splitter.split(allfeats, metadata['target']))[0]
train_ind, test_ind = splits

In [183]:
Xtrain = np.array(allfeats[train_ind].as_array().tolist())
Ytrain = np.array(metadata['target'][train_ind].tolist())

Xtest  = np.array(allfeats[test_ind].as_array().tolist())
Ytest  = np.array(metadata['target'][test_ind].tolist())

In [184]:
ncols = len(new_names)
npca  = (ncols  - 3)//len(pbnames)  + 3

In [185]:
pca = PCA(n_components=npca, whiten=True, svd_solver="full", random_state=42)
Xtrain_pca = pca.fit_transform(Xtrain)
Xtest_pca = pca.transform(Xtest)

In [186]:
def plasticc_log_loss(y_true, y_pred, relative_class_weights=None):
        """
        Implementation of weighted log loss used for the Kaggle challenge
        """
        predictions = y_pred.copy()

        # sanitize predictions
        epsilon = sys.float_info.epsilon # this is machine dependent but essentially prevents log(0)
        predictions = np.clip(predictions, epsilon, 1.0 - epsilon)
        predictions = predictions / np.sum(predictions, axis=1)[:, np.newaxis]

        predictions = np.log(predictions)
        # multiplying the arrays is equivalent to a truth mask as y_true only contains zeros and ones
        class_logloss = []
        for i in range(predictions.shape[1]):
            # average column wise log loss with truth mask applied
            result = np.average(predictions[:, i][y_true[:, i] == 1])
            class_logloss.append(result)
        return -1 * np.average(class_logloss, weights=relative_class_weights)
    

# Deep Learning First try

In [187]:
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F

In [188]:
Xtrain = np.array(allfeats[train_ind].as_array().tolist())
Ytrain = np.array(metadata['target'][train_ind].tolist())

Xtest  = np.array(allfeats[test_ind].as_array().tolist())
Ytest  = np.array(metadata['target'][test_ind].tolist())

display(Xtrain.shape, Ytrain.shape, Xtest.shape, Ytest.shape)

# map target values, e.g. 90->1...
for j, y in enumerate([Ytrain, Ytest]):
    unique_y = np.unique(y)
    class_map = dict()
    for i,val in enumerate(unique_y):
        class_map[val] = i

    y_map = np.zeros((y.shape[0],))
    if j == 0:
        Ytrain = np.array([class_map[val] for val in y])
    else:
        Ytest = np.array([class_map[val] for val in y])
Ytest, Ytrain, Ytrain[list(np.where(Ytrain == 14)[0])] # no class 14 in training set

(5493, 69)

(5493,)

(2355, 69)

(2355,)

(array([1, 3, 3, ..., 4, 4, 9]),
 array([11,  3, 11, ...,  2,  8,  9]),
 array([], dtype=int64))

In [189]:
counts = Counter(Ytrain)
labels, values = zip(*sorted(counts.items(), key=itemgetter(1)))
fig, ax = plt.subplots()

cmap = plt.cm.tab20
nlines = len(labels)
classcolor =  list(cmap(np.linspace(0,1,nlines)))[::-1]

# we'll create a mapping between class and color
classcolmap = dict(zip(labels, classcolor))

indexes = np.arange(nlines)
width = 1
ax.bar(indexes, values, width, edgecolor='k',\
       linewidth=1.5, tick_label=labels, log=True, color=classcolor)
ax.set_xlabel('Target')
ax.set_ylabel('Number of Objects')
fig.tight_layout()

<IPython.core.display.Javascript object>

In [190]:
{6: 0, 15: 1, 16: 2, 42: 3, 52: 4, 53: 5, 62: 6, 64: 7, 65: 8, 67: 9, 88: 10, 90: 11, 92: 12, 95: 13}

{6: 0,
 15: 1,
 16: 2,
 42: 3,
 52: 4,
 53: 5,
 62: 6,
 64: 7,
 65: 8,
 67: 9,
 88: 10,
 90: 11,
 92: 12,
 95: 13}

In [191]:
import torch.utils.data as data_utils

XtrainTensor, YtrainTensor  = torch.Tensor(Xtrain),torch.Tensor(Ytrain)
XtestTensor, YtestTensor = torch.Tensor(Xtest),torch.Tensor(Ytest)

train = data_utils.TensorDataset(XtrainTensor, YtrainTensor)
train_loader = data_utils.DataLoader(train, batch_size=16, shuffle=True)
test = data_utils.TensorDataset(XtestTensor, YtestTensor)
test_loader = data_utils.DataLoader(test)

In [192]:
# Benos classifier

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 2 fully connected layers
        
        self.fc1 = nn.Linear(69,69)
        self.fc2 = nn.Linear(69,50)
        self.fc3 = nn.Linear(50,15)
        

    def forward(self, x):
        # Max pooling over a (2, 2) window
        #x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        # If the size is a square you can only specify a single number
        
        # Flatten 2d array to pass to linear layer, -1 works for arbitrary #rows which can vary with batchsize
        #x = x.view(-1, self.num_flat_features(x))
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
    prediction_list = []
    def predict(self, dataloader):
        for i, batch in enumerate(dataloader):
            pred, output = self.forward(batch)
            prediction_list.append(pred.cpu())
        return prediction_list


net = Net()
print(net)

Net(
  (fc1): Linear(in_features=69, out_features=69, bias=True)
  (fc2): Linear(in_features=69, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=15, bias=True)
)


In [193]:
# training
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [194]:
for epoch in range(200):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, sample in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = sample

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    if epoch % 100 == 0:    # print every 2000 mini-batches
        # validation loss
        prediction_list = []
        running_loss_test = 0.0
        for i_test, sample_test in enumerate(test_loader, 0):
            # get the inputs
            inputs_test, labels_test = sample_test

            # zero the parameter gradients
            optimizer.zero_grad()

            outputs_test = net(inputs_test)
            loss_test = criterion(outputs_test, labels_test.long())
            prediction_list.append(np.argmax(outputs_test.detach().numpy()))
            
            running_loss_test += loss_test.item()
            
        print('[%d, %5d] loss: %.3f validation_loss: %.3f' % \
              (epoch, i, running_loss / (len(train_loader)*100), loss_test/len(test_loader)))
        running_loss = 0.0

print('Finished Training')

[0,   343] loss: 0.058 validation_loss: 0.001
[100,   343] loss: 0.022 validation_loss: 0.002
Finished Training


# Accuracy

In [195]:
Ypred = np.array(prediction_list)
{"Accuracy in %": np.round(1-len((Ytest-Ypred).nonzero()[0])/(len(Ytest))*1,4)}

{'Accuracy in %': 0.296}

In [196]:
# we didn't release the weights with the contest at Kaggle's request
# they have been probed through the leaderboard though
# we leave the reader to adjust this as they see fit
# the official metric is always what is on Kaggle's leaderboard. 
# This notebook is solely for demonstration.
weights = np.ones(nlines)


# we want the actual prediction probabilities

# we also need to express the truth table as a matrix
sklearn_truth = np.zeros((len(Ytest), nlines))
label_index_map = {6: 0, 15: 1, 16: 2, 42: 3, 52: 4, 53: 5, 62: 6, 64: 7, 65: 8, 67: 9, 88: 10, 90: 11, 92: 12, 95: 13}
for i, x in enumerate(Ytest):
    sklearn_truth[i][Ytest[i]] = 1
 
      
logloss = plasticc_log_loss(sklearn_truth, Ypred, relative_class_weights=weights)
print("LogLoss: {:.3f}".format(logloss))

AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
cm = confusion_matrix(Ytest, Ypred, labels=labels)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
annot = np.around(cm, 2)
fig, ax = plt.subplots(figsize=(9,7))
sns.heatmap(cm, xticklabels=labels, yticklabels=labels, cmap='Blues', annot=annot, lw=0.5)
ax.set_xlabel('Predicted Label')
ax.set_ylabel('True Label')
ax.set_aspect('equal')